In [ ]:
import sdf_helper as sh
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.fftpack import fft

# base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch2d/垂直激光5GeV+bx1e10T'
base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch2d/激光电子对撞/3pi线偏振'
file_prefix = "field_FFT"
file_suffix = ".sdf"

# file_index = 5
# file_name = f"{file_prefix}{file_index:04d}{file_suffix}"
# file_path = os.path.join(base_path, file_name)
# Field_Data = sh.getdata(file_path)
# sh.list_variables(Field_Data)

start_index = 0
end_index = 8000  # 共2001帧
dt_fs = 0.02       # 时间间隔 0.01 fs
dt = dt_fs * 1e-15  # 转为秒

# === 准备电场序列容器 ===
ey, ex, ez = [], [], []

# === 读取数据 ===
def read_field_generator(base_path, file_prefix, file_suffix, start_index, end_index):
    for file_index in range(start_index, end_index + 1):
        file_name = f"{file_prefix}{file_index:04d}{file_suffix}"
        file_path = os.path.join(base_path, file_name)
        Field_Data = sh.getdata(file_path)
        ex_val = Field_Data.Electric_Field_Ex_Core_TT.data[750, 300]
        ey_val = Field_Data.Electric_Field_Ey_Core_TT.data[750, 300]
        ez_val = Field_Data.Electric_Field_Ez_Core_TT.data[750, 300]
        # ex_val = Field_Data.Electric_Field_Ex.data[1500, 750]
        # ey_val = Field_Data.Electric_Field_Ey.data[1500, 750]
        # ez_val = Field_Data.Electric_Field_Ez.data[1500, 750]
        yield ex_val, ey_val, ez_val

# === 填充电场序列 ===
for ex_val, ey_val, ez_val in read_field_generator(base_path, file_prefix, file_suffix, start_index, end_index):
    ex.append(ex_val)
    ey.append(ey_val)
    ez.append(ez_val)

# === 转为 NumPy 数组 ===
ex_array = np.array(ex)
ey_array = np.array(ey)
ez_array = np.array(ez)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
# from numpy import exp, conjugate, real, imag, sqrt, arctan, tan, pi
# 假设已给定时间间隔 dt (秒) 和电场时间序列 ex_array, ey_array, ez_array
dt = dt_fs * 1e-15  # 飞秒转秒
ex_array = np.array(ex)
ey_array = np.array(ey)
ez_array = np.array(ez)

N = len(ey_array)

# 傅里叶变换
fft_ex = fft(ex_array)
fft_ey = fft(ey_array)
fft_ez = fft(ez_array)

# 频率轴
freqs = fftfreq(N, d=dt)

# 只取正频率部分
mask = freqs >= 0
freqs_pos = freqs[mask]

fft_ex_pos = fft_ex[mask] / N
fft_ey_pos = fft_ey[mask] / N
fft_ez_pos = fft_ez[mask] / N

# 补偿能量
fft_ex_pos[1:-1] *= 2
fft_ey_pos[1:-1] *= 2
fft_ez_pos[1:-1] *= 2

# # 绘图：椭圆度随频率变化
# plt.figure(figsize=(8,5))
# plt.plot(w_x[0:1000] * 2 * pi, np.abs(tan(chi[0:1000])), 'k', label='Ellipticity')
# plt.plot(freqs_pos*1e-12, np.abs(np.tan(chi)), 'k', label='Ellipticity')

In [ ]:
# 计算斯托克斯参数（以 Ey 和 Ez 计算偏振度）
S0 = np.abs(fft_ey_pos)**2 + np.abs(fft_ez_pos)**2
S1 = np.abs(fft_ey_pos)**2 - np.abs(fft_ez_pos)**2
S2 = 2 * np.real(fft_ey_pos * np.conj(fft_ez_pos))
S3 = 2 * np.imag(fft_ey_pos * np.conj(fft_ez_pos))

S1 = S1.astype(np.float128)
S2 = S2.astype(np.float128)
S3 = S3.astype(np.float128)
S0 = S0.astype(np.float128)

S01 = np.sum(S0)
S11 = np.sum(S1)
S21 = np.sum(S2)
S31 = np.sum(S3)

# === 偏振度计算 ===
P1 = np.sqrt(S11**2 + S21**2 + S31**2) / S01       # 总偏振度
P_lin1 = np.sqrt(S11**2 + S21**2) / S01           # 线偏振度
P_circ1 = S31 / S01                              # 圆偏振度

# === 输出结果 ===
print("=== 偏振度分析结果 ===")
print(f"总偏振度     P      = {P1:.4f}")
print(f"线偏振度     P_lin  = {P_lin1:.4f}")
print(f"圆偏振度     P_circ = {P_circ1:.4f}")

In [ ]:
# 计算偏振度参数
P = np.sqrt(S1**2 + S2**2 + S3**2) / S0        # 总偏振度
P_lin = np.sqrt(S1**2 + S2**2) / S0            # 线偏振度
P_circ = S3 / S0                               # 圆偏振度
P_horiz = np.abs(fft_ey_pos)**2 / S0           # 水平偏振强度占比
P_vert = np.abs(fft_ez_pos)**2 / S0            # 垂直偏振强度占比
# 椭圆角 chi
chi = 0.5 * np.arctan(S3 / np.sqrt(S1**2 + S2**2))
# 椭圆率 = tan(chi)
ellipticity = np.tan(chi)

In [ ]:
# 1. 总偏振度折线图 + 点图
plt.figure(figsize=(8,5))
plt.scatter(freqs_pos * 1e-12, P, s=10, color='r', label='Degree of Polarization')
# plt.plot(freqs_pos*1e-12, P, 'k-', label='Degree of Polarization')
plt.title('Degree of Polarization')
plt.xlabel('Frequency (THz)')
plt.ylabel('P')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()
print(P)


In [ ]:
# 2. 线偏振度
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, P_lin, 'b-', label='Linear Polarization')
plt.title('Linear Polarization Degree')
plt.xlabel('Frequency (THz)')
plt.ylabel('P_lin')
# plt.xlim(0,2000)
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 3. 圆偏振度
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, abs(P_circ), 'g-', label='Circular Polarization')
# plt.scatter(freqs_pos * 1e-12, P_circ, s=10, color='r', label='Circular Polarization')
plt.title('Circular Polarization Degree')
plt.xlabel('Frequency (THz)')
plt.ylabel('P_circ')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 4. 水平偏振占比
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, P_horiz, 'm-', label='Horizontal Polarization Fraction')
plt.title('Horizontal Polarization')
plt.xlabel('Frequency (THz)')
plt.ylabel('Fraction')
# plt.xlim(0,2000)
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 5. 垂直偏振占比
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, P_vert, 'c-', label='Vertical Polarization Fraction')
plt.title('Vertical Polarization')
plt.xlabel('Frequency (THz)')
plt.ylabel('Fraction')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 6. Ey 和 Ez 频谱图（幅度）
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, np.abs(fft_ey_pos), label='Ey Amplitude')
# plt.plot(freqs_pos*1e-12, np.abs(fft_ez_pos), label='Ez Amplitude')
plt.title('Frequency Spectrum')
plt.xlabel('Frequency (THz)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# 计算椭圆角 chi
chi = 0.5 * np.arctan2(S3, np.sqrt(S1**2 + S2**2))

# 椭圆率 = tan(chi)
ellipticity = np.tan(chi)

# 绘制椭圆率随频率变化图
plt.figure(figsize=(8,5))
plt.plot(freqs_pos*1e-12, np.abs(ellipticity), 'k-', label='Ellipticity |tan(chi)|')
plt.title('Ellipticity vs Frequency')
plt.xlabel('Frequency (THz)')
plt.ylabel('Ellipticity |tan(chi)|')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 计算方位角 psi，单位为弧度
psi = 0.5 * np.arctan2(S2, S1)

# 若需要度数，转换如下
psi_deg = np.degrees(psi)

plt.figure(figsize=(8,5))
plt.scatter(freqs_pos * 1e-12, psi_deg, color='b', s=10, label='Orientation Angle (degrees)')
plt.title('Orientation Angle vs Frequency')
plt.xlabel('Frequency (THz)')
plt.ylabel('Orientation Angle (degrees)')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# === 频谱总积分（可视为全频带功率）===


In [ ]:
# 上交大
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from numpy import exp, conjugate, real, imag, sqrt, arctan, tan, pi

# 傅里叶变换 y 和 z 分量电场
sp_x = np.fft.fft(ex_array)  # Ey电场傅里叶变换
sp_y = np.fft.fft(ey_array)  # Ey电场傅里叶变换
sp_z = np.fft.fft(ez_array)  # Ez电场傅里叶变换

# 计算时间间隔
dt = dt_fs * 1e-15  # 飞秒转秒

# 频率坐标（沿传播方向 x）
w_x = np.fft.fftfreq(len(sp_x), d=dt)
w_y = np.fft.fftfreq(len(sp_y), d=dt)
w_z = np.fft.fftfreq(len(sp_z), d=dt)

# 计算斯托克斯参数（用 y 和 z 方向频谱）
S0 = sp_y * conjugate(sp_y) + sp_z * conjugate(sp_z)  # 总强度
S1 = sp_y * conjugate(sp_y) - sp_z * conjugate(sp_z)  # 强度差
S2 = 2 * real(sp_y * conjugate(sp_z))                 # 相干实部
S3 = 2 * imag(sp_y * conjugate(sp_z))                 # 相干虚部

# 计算偏振度
V = sqrt(S1**2 + S2**2 + S3**2) / S0
chi = 0.5 * arctan(S3 / sqrt(S1**2 + S2**2))

# # 绘图：椭圆度随频率变化
plt.figure(figsize=(8,5))
# plt.plot(w_x[0:1000] * 2 * pi, np.abs(tan(chi[0:1000])), 'k', label='Ellipticity')
plt.plot(w_x*1e-12, np.abs(np.tan(chi)), 'k', label='Ellipticity')
plt.xlim(0,25000)
# # 找峰值并标注
peaks, properties = find_peaks(np.abs(tan(chi)), height=0)
peak_heights = properties['peak_heights']
# peak_positions = w_x[peaks] * 2 * pi
peak_positions = w_x[peaks]*1e-12
e_prime = np.mean(peak_heights)

plt.scatter(peak_positions, peak_heights, color='r', s=30, marker='D', label='Maxima')

# plt.xlim(0, 200)
plt.title('(c) Ellipticity for propagation along x, polarization y-z', loc='left', fontsize=13)
plt.xlabel(r'$\omega / \omega_L$', fontsize=13)
plt.xlabel(r'Frequency (Hz)', fontsize=13)
plt.ylabel('Ellipticity', fontsize=13)
plt.legend()
plt.tight_layout()
plt.show()